In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Variable_Calculation/CalculateMoreVariables'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data,parcel] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
# print('Loading In Data')
# u_data=data['u'].interp(xf=data['xh']).data
# v_data=data['v'].interp(yf=data['yh']).data
# print('done')


# Nt=len(data['time'])
# for t in np.arange(Nt):
#     u_data=data['u'].isel(time=t).interp(xf=data['xh']).data
#     v_data=data['v'].isel(time=t).interp(yf=data['yh']).data
#     print('calculating convergence and taking mean')
#     conv=-(Ddx_3D(u_data,1000)+Ddy_3D(v_data,1000))

In [ ]:
if res=='1km':
    dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
if res=='250m':
    dir2='/mnt/lustre/koa/scratch/air673/'
def initiate_array():
    # Define array dimensions (adjust based on your data)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file = dir2 + 'Variable_Calculation/OUTPUT/' + 'Convergence'+f'_{res}_{t_res}'+'.h5'
    
    with h5py.File(out_file, 'a') as f:
        # Check if the dataset 'theta_e' already exists
        if 'conv' not in f:
            # Create a dataset with the full size for all time steps (initially empty)
            f.create_dataset('conv', 
                             (t_size, z_size, y_size, x_size),  # Full size for all timesteps
                             maxshape=(None, z_size, y_size, x_size),  # Unlimited timesteps (can grow along time dimension)
                             dtype='float64', 
                             chunks=(1, z_size, y_size, x_size))  # Chunks for time axis to allow resizing

            
def add_timestep_at_index(timestep_data, index):
    out_file = dir2 + 'Variable_Calculation/OUTPUT/' + 'Convergence'+f'_{res}_{t_res}'+'.h5'
    with h5py.File(out_file, 'a') as f:
        # Access the existing dataset 'theta_e'
        dataset = f['conv']
        
        # Assign the new timestep data at the specified index
        dataset[index] = timestep_data

In [ ]:
####################################
#RUNNING

In [ ]:
if res=='1km':
    dx=1000
    dy=1000
elif res=='250m':
    dx=250
    dy=250

In [ ]:
#MAKING ARRAY TO STORE THETA_E
initiate_array()

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')

    u_data=data['uinterp'].isel(time=t).data
    v_data=data['vinterp'].isel(time=t).data
    print('calculating convergence and taking mean')
    conv=-(Ddx_3D(u_data,1000)+Ddy_3D(v_data,1000))
    add_timestep_at_index(conv, t)

In [ ]:
######################################################

In [ ]:
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# file_path = dir2 + 'Variable_Calculation/' + 'Convergence' + f'_{res}_{t_res} + '.h5'
# with h5py.File(file_path, 'r') as f:
#     Conv = f['conv'][:]

# plt.contourf(Conv[300,10])
# plt.colorbar()